
# Instruction and Results: ClusterOmedianIZE Algorithm for $1 \mid p_j =p \mid TE$

This notebook is part of the research based on paper _Migration from Sequence to Schedule in Total Earliness and Tardiness Scheduling Problem_. ([here](http://www.optimization-online.org/DB_HTML/2019/07/7280.html)).

> This notebook runs on Python 2.

---



In [22]:
from collections import OrderedDict
import numpy as np
# new function
def detectChainFirst(nbJobs,due,s,pr):
    c = 0
    chain = []
    orphanedJob = 0
    for j in range(1,nbJobs):
        orphanedJob = 1 # means one job exists in the chain
        if (due[j] - due[j-1] <= pr):
            orphanedJob = 0
            chain.append((c,j-1))
            chain.append((c,j))
        elif orphanedJob == 1:
            chain.append((c,j-1))
            c = c + 1
    chain.append((0,0))
    if (c,nbJobs-1) not in chain:
        chain.append((c,nbJobs-1))
    return list(OrderedDict.fromkeys(chain))
# new function
def isJobsStraddle(chain,s,pr):
    chainInd,jobInd = zip(*chain) # unzip a list of tuple
    for c in range(max(chainInd)+1):
        jobIndPerChain = [jobInd[i] for i in range(len(jobInd)) if chainInd[i]==c]
        if s[jobIndPerChain[-1]] - s[jobIndPerChain[0]] < (len(jobIndPerChain)-1)*pr:
            return True
            break
# new function
def startCorrect(chain,s,pr,t,due):
    chainInd,jobInd = zip(*chain) # unzip a list of tuple
    for c in range(max(chainInd)+1):
        jobIndPerChain = [jobInd[i] for i in range(len(jobInd)) if chainInd[i]==c]
        if s[jobIndPerChain[-1]] - s[jobIndPerChain[0]] < (len(jobIndPerChain)-1)*pr:
            s[jobIndPerChain[0]] = np.maximum(t, int(np.median([due[jobIndPerChain[j]]-(j+1)*pr for j in range(len(jobIndPerChain))])))
            for j in jobIndPerChain:
                if j!=jobIndPerChain[0]:
                    s[j] = s[jobIndPerChain[0]] + (j-jobIndPerChain[0])*pr
    return s
# new function
def reDetectChain(nbJobs,s,pr):
    c = 0
    chain = []
    orphanedJob = 0
    for j in range(nbJobs-1):
        orphanedJob = 1 # means one job exists in the chain
        if s[j] + pr >= s[j+1]:
            orphanedJob = 0
            chain.append((c,j))
            chain.append((c,j+1))
        elif orphanedJob == 1:
            chain.append((c,j))
            c = c + 1
    chain.append((0,0))
    if (c,nbJobs-1) not in chain:
        chain.append((c,nbJobs-1))
    return list(OrderedDict.fromkeys(chain))
# new function
def objEarlinessTardinessTotal(nbJobs,s,due,pr):
    return sum(np.absolute(s[j]+pr-due[j]) for j in range(nbJobs))

In [23]:
def ClusterOmedianIZE(nbJobs,s,pr,t,due):
# sort based on EDD rule
    chain = detectChainFirst(nbJobs,due,s,pr)
    while isJobsStraddle(chain,s,pr):
        s = startCorrect(chain,s,pr,t,due)
        chain = reDetectChain(nbJobs,s,pr)
    return s

In [24]:
# proc = 5
# due = [15,16,18,33,35]

proc = 5
due = [ 87, 285, 190, 122, 187, 253, 183, 129, 242, 194, 179, 254, 121]

# proc = 5;
# due  = [ 87, 285, 190, 122, 187, 253, 183, 129, 242, 194, 179, 254, 121,
#        287, 184,  75, 228,  70, 151, 196, 262, 189, 284, 206, 207, 192,
#        100, 118, 265, 265, 283, 291, 297, 272, 146, 136, 191, 283, 187,
#         57, 113, 111,  72, 107,  51, 178, 110, 259,  58, 266, 191, 165,
#        225, 284, 171, 250,  80, 121, 181, 248, 199,  99, 107,  53, 246,
#         74, 291,  93, 126,  76, 102, 130, 159, 165,  91, 260,  65, 114,
#        246,  75, 161, 276, 265, 185,  76, 203, 154,  72,  59, 245, 281,
#        176,  73, 175, 150, 205, 215, 107, 133, 216];



nbJobs = len(due)
job = range(nbJobs)
due = sorted([due[j] for j in job])
s = [j-proc for j in due]
s = ClusterOmedianIZE(nbJobs,s,proc,0,due)
print( s)
print (objEarlinessTardinessTotal(nbJobs,s,due,proc))

[82, 114, 119, 124, 172, 177, 182, 187, 192, 237, 246, 251, 280]
16
